In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
os.chdir(".")
# from parc.data import EnergeticMatDataPipeLine as EmData
from parc import misc, metrics, model,visualization
from parc.model import model_burgers
from skimage.measure import block_reduce



2024-01-18 11:09:56.333802: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data pipeline

In [2]:
import time
import os
import numpy as np
import skimage
from skimage.measure import block_reduce


Re_list = [15,20,30,40,60,80,100,120,140,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000]

def clip_raw_data():
    data_whole = []
    # r_whole = []
    for Re in Re_list:
        data_file_name = 'Re_' + str(int(Re)) + '.npy'
#         print(data_file_name)
        file_path = './ns_data/' + data_file_name                
        if os.path.exists(file_path):
            raw_data = np.float32(np.load(file_path))
            raw_data = np.expand_dims(raw_data, axis = 0)
#             raw_data = skimage.measure.block_reduce(raw_data[:,:,:,:], (1,2,2,1),np.max)
            data_shape = raw_data.shape
            # print(data_shape)
            # norm_r = Re/1000
            # r_img = norm_r*np.ones(shape = (data_shape[0],data_shape[1],data_shape[2],1))
            # r_whole.extend(r_img)

            data_whole.extend(raw_data)

    data_whole = np.concatenate([data_whole], axis=0)
    # r_whole = np.concatenate([r_whole], axis=0)
    return data_whole

seq_clipped = clip_raw_data()



In [3]:
# Normalization
def data_normalization(input_data,no_of_channel):
    norm_data = np.zeros(input_data.shape)
    min_val = []
    max_val = []
    for i in range(no_of_channel):
        iter_max_val = np.amax(input_data[:,:,:,i::3])
        iter_min_val = np.amin(input_data[:,:,:,i::3])
        norm_data[:,:,:,(i)::no_of_channel] = ((input_data[:,:,:,(i)::no_of_channel] - iter_min_val)) / (iter_max_val - iter_min_val + 1E-9)
        min_val.append(iter_min_val)
        max_val.append(iter_max_val)
    return norm_data, min_val, max_val

In [4]:
seq_clipped.shape
seq_norm = data_normalization(seq_clipped, 3)

In [6]:
print(seq_norm[1], seq_norm[2])


[-2.757973, -2.6737006, -255955.38] [4.0362315, 2.584277, 109451.57]


In [6]:
Re_list = [15,20,30,40,60,80,100,120,140,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000]
train_list = [30,40,80,100,150,200,250,300,400,450,500,600,650,700,800,850,900,950]
test_list = [20,60,140,350,550,750,1000]
idx = 0
train_idx =[]
test_idx =[]
for Re in Re_list:
    if Re in train_list:
        train_idx.append(idx)
    elif Re in test_list:
        test_idx.append(idx)
    idx += 1

In [8]:
print(test_idx)

[1, 4, 8, 13, 17, 21, 26]


In [7]:
train_seq = [seq_norm[0][idx:idx+1,:,:,:] for idx in train_idx]
# train_re =[seq_clipped[1][idx:idx+1,:,:,:] for idx in train_idx]

test_seq = [seq_norm[0][idx:idx+1,:,:,:] for idx in test_idx]
# test_re =[seq_clipped[1][idx:idx+1,:,:,:] for idx in test_idx]


In [8]:
train_seq = np.concatenate(train_seq, axis = 0)
# train_re = np.concatenate(train_re, axis = 0)

test_seq = np.concatenate(test_seq, axis = 0)
# test_re = np.concatenate(test_re, axis = 0)
print(test_seq.shape)

(7, 128, 256, 117)


In [11]:
print(train_seq.shape)

(18, 128, 256, 117)


In [12]:
def create_train_data(seq, no_of_fields, sequence_length = 2):
    shape = seq.shape
    num_time_steps = np.int32((shape[-1]-1)/3)
    vel_seq_whole = []
    # re_seq_whole = []
#     constant_whole = []
    for i in range(shape[0]):
#         constant = seq[i:i+1,:,:,0:1]
        for j in range(num_time_steps-sequence_length+1):
            vel_seq_case = np.expand_dims(seq[i, :, :, (j*no_of_fields):(j*no_of_fields+sequence_length*no_of_fields)],axis = 0)
#             vel_seq_case = np.concatenate([constant,vel_seq_case],axis = -1)
            vel_seq_whole.extend(vel_seq_case)
            # re_seq_whole.extend(re[i:i+1,:,:,:])
    vel_seq_whole = np.concatenate([vel_seq_whole], axis=0)
    # re_seq_whole = np.concatenate([re_seq_whole], axis=0)

    return vel_seq_whole

train_data = create_train_data(train_seq, no_of_fields = 3, sequence_length = 13)

# Training


### Stage 1: Differentiator training

In [13]:
# Create tf.dataset
dataset_input = tf.data.Dataset.from_tensor_slices(train_data[:,:,:,:3])
dataset_label = tf.data.Dataset.from_tensor_slices(train_data[:,:,:,3:])
dataset = tf.data.Dataset.zip((dataset_input, dataset_label))
dataset = dataset.shuffle(buffer_size = 798) 
dataset = dataset.batch(4)

2024-01-06 14:52:11.572250: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78791 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:0f:00.0, compute capability: 8.0


In [ ]:
tf.keras.backend.clear_session()
parc = PARCv2_ns(n_time_step = 12, step_size= 1/38, solver = "heun", mode = "differentiator_training")
parc.differentiator.load_weights('parc2_diff_ns_heun_10_2.h5')
parc.poisson.load_weights('parc2_poisson_ns_heun_10_2.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.000001, beta_1 = 0.5, beta_2 = 0.999))
parc.fit(dataset, epochs = 50, shuffle = True)

In [15]:
parc.differentiator.save_weights('parc2_diff_ns_heun_12.h5')
parc.poisson.save_weights('parc2_poisson_ns_heun_12.h5')

In [ ]:
tf.keras.backend.clear_session()
parc = PARCv2_ns(n_time_step = 12, step_size= 1/38, solver = "heun", mode = "integrator_training")
parc.differentiator.load_weights('parc2_diff_ns_heun_12.h5')
parc.poisson.load_weights('parc2_poisson_ns_heun_12.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.000001, beta_1 = 0.5, beta_2 = 0.999))
parc.fit(dataset, epochs = 50, shuffle = True)

In [27]:
parc.integrator.save_weights('parc2_int_ns_heun_12.h5')

# Validation

In [16]:
tf.keras.backend.clear_session()
parc = PARCv2_ns(n_time_step = 37, step_size= 1/38, solver = "heun", use_data_driven_int = False, mode = "differentiator_training")
parc.differentiator.load_weights('parc2_diff_ns_heun_12.h5')
parc.poisson.load_weights('parc2_poisson_ns_heun_12.h5')
parc.integrator.load_weights('parc2_int_ns_heun_10.h5')
parc.compile()
parc.build(input_shape = (None,128,256,3))

In [17]:
parc.summary()

Model: "par_cv2_ns"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_2 (Functional)        (None, 128, 256, 2)       15187522  
                                                                 
 model_3 (Functional)        (None, 128, 256, 1)       229313    
                                                                 
 model_4 (Functional)        (None, 128, 256, 2)       439042    
                                                                 
Total params: 15855879 (60.49 MB)
Trainable params: 15416835 (58.81 MB)
Non-trainable params: 439044 (1.67 MB)
_________________________________________________________________


In [18]:
parc.save('parc2_ns_heun.keras')

In [19]:
loaded_parc = tf.keras.models.load_model('parc2_ns_heun.keras')

In [10]:
pred_whole =[]
for idx in range(7):
    input_seq_current = tf.cast(test_seq[idx:idx+1,:,:,:3], dtype = tf.float32)
    output = loaded_parc.predict(input_seq_current)
    pred_whole.append(output)
pred = np.concatenate(pred_whole,axis = 0)

2024-01-07 10:45:56.534022: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2024-01-07 10:45:57.343827: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:625] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1/1 [==============================] - 0s 248ms/step


In [70]:
def DeNormalization(y_pred, min_val, max_val, no_of_channel):
    denorm_data = np.zeros(y_pred.shape)
    
    for i in range(no_of_channel):
        print(max_val[i], min_val[i])
        denorm_data[:,:,:,(i)::no_of_channel] = (y_pred[:,:,:,(i)::no_of_channel] * (max_val[i] - min_val[i])) + min_val[i]
    return denorm_data

y_pred_denorm = DeNormalization(pred,seq_norm[1], seq_norm[2], no_of_channel = 3)
# gt_denorm = DeNormalization(test_seq,seq_norm[1], seq_norm[2], no_of_channel = 3)
# print(np.amax(np.sqrt(gt_denorm[3,:,:,0::3]**2 + gt_denorm[3,:,:,1::3]**2)))

4.0362315 -2.757973
2.584277 -2.6737006
109451.57 -255955.38


In [74]:
np.save('./plotting/ns/parcv2_ns.npy',y_pred_denorm)